In [ ]:
# Copyright 2021-2022 Boris Shminke
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
# saturation prover environment can be created
# as any other OpenAI Gym environment
import gym
import os
from glob import glob
import sys
# this example uses package's resources
# one can indicate the path to unpacked (TPTP)[http://tptp.org/] instead
# it's obligatory to set the list of problems
# Here are CNF problems from TPTP
if sys.version_info.major == 3 and sys.version_info.minor >= 9:
    from importlib.resources import files
else:
    from importlib_resources import files

problem_list = sorted(glob(
    os.path.join(
        files(
            "gym_saturation"
        ).joinpath(os.path.join(
            "resources", "TPTP-mock"
        ),
        os.path.join("Problems", "*", "*-*.p")
    )
)))
env = gym.make(
    "GymSaturation-v0",
    problem_list=problem_list
)

In [2]:
# to make results reproducible, let's set the seed
print(env.seed(2))

2


In [3]:
# parsing is done with `lark` package which sometimes might be slow
# reimplementing parsing in a faster language than Python might help
obs = env.reset()

In [4]:
# during the reset a problem is chosen at random
print(env.render())

cnf(b_equals_bb, hypothesis, equal_sets(b, bb)).
cnf(element_of_b, hypothesis, member(element_of_b, b)).
cnf(prove_element_of_bb, negated_conjecture, ~member(element_of_b, bb)).
cnf(membership_in_subsets, axiom, ~member(X0, X1) | ~subset(X1, X2) | member(X0, X2)).
cnf(subsets_axiom1, axiom, subset(X4, X3) | member(member_of_1_not_of_2(X4,X3), X4)).
cnf(subsets_axiom2, axiom, ~member(member_of_1_not_of_2(X6,X5), X5) | subset(X6, X5)).
cnf(set_equal_sets_are_subsets1, axiom, ~equal_sets(X8, X7) | subset(X8, X7)).
cnf(set_equal_sets_are_subsets2, axiom, ~equal_sets(X9, X10) | subset(X10, X9)).
cnf(subsets_are_set_equal_sets, axiom, ~subset(X12, X11) | ~subset(X11, X12) | equal_sets(X11, X12)).


In [5]:
# observation is a dict with two keys: "action_mask" and "real_obs"
# "action_mask" is 1 for actions an agent can choose and 0 for the rest
# "real_obs" is a JSON representation of logical clauses
print(obs.keys())
print(obs["action_mask"][:20])
print(obs["real_obs"][0])

dict_keys(['real_obs', 'action_mask'])
[1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
b'{"literals":[{"negated":false,"atom":{"name":"equal_sets","arguments":[{"name":"b","arguments":[]},{"name":"bb","arguments":[]}]}}],"label":"b_equals_bb","role":"hypothesis","inference_parents":[],"inference_rule":null,"processed":false,"birth_step":0}'


In [6]:
 # possible actions are indices of not processed clauses from the state
# formally, action space is a large enough discrete
print(env.action_space)
# but the number of ones in the mask
print(obs["action_mask"].sum())
# equals to number of not processed clauses
print(sum(
    [1.0 for clause in obs["real_obs"] if b'"processed": True' not in clause]
))

Discrete(100000)
9.0
9.0


In [7]:
# only "ansi" and "human" rendering methods are implemented
# "human" (default) returns TPTP representation
# "ansi" returns a JSON string representing the state
print(env.render("ansi")[:1000])

[b'{"literals":[{"negated":false,"atom":{"name":"equal_sets","arguments":[{"name":"b","arguments":[]},{"name":"bb","arguments":[]}]}}],"label":"b_equals_bb","role":"hypothesis","inference_parents":[],"inference_rule":null,"processed":false,"birth_step":0}', b'{"literals":[{"negated":false,"atom":{"name":"member","arguments":[{"name":"element_of_b","arguments":[]},{"name":"b","arguments":[]}]}}],"label":"element_of_b","role":"hypothesis","inference_parents":[],"inference_rule":null,"processed":false,"birth_step":0}', b'{"literals":[{"negated":true,"atom":{"name":"member","arguments":[{"name":"element_of_b","arguments":[]},{"name":"bb","arguments":[]}]}}],"label":"prove_element_of_bb","role":"negated_conjecture","inference_parents":[],"inference_rule":null,"processed":false,"birth_step":0}', b'{"literals":[{"negated":true,"atom":{"name":"member","arguments":[{"name":"X0"},{"name":"X1"}]}},{"negated":true,"atom":{"name":"subset","arguments":[{"name":"X1"},{"name":"X2"}]}},{"negated":false

In [8]:
# the package includes a simple episode function
from gym_saturation.agent_testing import episode
# and an agent selecting the shortest clause five times
# and then once --- the oldest clause
from gym_saturation.agent_testing import SizeAgeAgent

env.unwrapped.problem_list = [
    files("gym_saturation")
    .joinpath(
        os.path.join("resources", "TPTP-mock", "Problems", "SET", "SET001-1.p")
    )
]
# this agent manages to solve a simple problem from TPTP
episode(env, SizeAgeAgent(5, 1))

In [9]:
# the proof here consists of only three steps
# but this agent finds it only after 11 steps
# a trained DL agent might have done better
print(env._elapsed_steps)

10


In [10]:
# one can now check the TSTP proof found
print(env.tstp_proof)

cnf(x7b469f0d_b821_11ec_a9ea_0f4925154247, lemma, ~subset(b, X2) | member(element_of_b, X2), inference(resolution, [], [element_of_b, membership_in_subsets])).
cnf(x7b469ee7_b821_11ec_a9ea_0f4925154247, lemma, subset(b, bb), inference(resolution, [], [b_equals_bb, set_equal_sets_are_subsets1])).
cnf(x7b469f96_b821_11ec_a9ea_0f4925154247, lemma, ~subset(b, bb), inference(resolution, [], [prove_element_of_bb, x7b469f0d_b821_11ec_a9ea_0f4925154247])).
cnf(x7b469feb_b821_11ec_a9ea_0f4925154247, lemma, $false, inference(resolution, [], [x7b469ee7_b821_11ec_a9ea_0f4925154247, x7b469f96_b821_11ec_a9ea_0f4925154247])).


In [11]:
# paramodulation test
env.unwrapped.problem_list = [
    files("gym_saturation")
    .joinpath(
        os.path.join("resources", "TPTP-mock", "Problems", "TST", "TST002-1.p")
    )
]
env.reset()
print(env.render())
episode(env, SizeAgeAgent(5, 1))
# here the proof consists of five steps
# but the agent needs ten
print(env.tstp_proof)
print(
    len(env.tstp_proof.split("\n")), 
    env._elapsed_steps
)

cnf(a1, hypothesis, q(a)).
cnf(a2, hypothesis, ~q(a) | f(X0) = X0).
cnf(a3, hypothesis, p(X1) | p(f(a))).
cnf(c, negated_conjecture, ~p(X2) | ~p(f(X2))).
cnf(x7c641060_b821_11ec_a9ea_0f4925154247, lemma, p(f(a)), inference(factoring, [], [a3])).
cnf(x7c6410a9_b821_11ec_a9ea_0f4925154247, lemma, f(X0) = X0, inference(resolution, [], [a1, a2])).
cnf(x7c641081_b821_11ec_a9ea_0f4925154247, lemma, ~p(a), inference(resolution, [], [x7c641060_b821_11ec_a9ea_0f4925154247, c])).
cnf(x7c64115f_b821_11ec_a9ea_0f4925154247, lemma, p(a), inference(paramodulation, [], [x7c641060_b821_11ec_a9ea_0f4925154247, x7c6410a9_b821_11ec_a9ea_0f4925154247])).
cnf(x7c641184_b821_11ec_a9ea_0f4925154247, lemma, $false, inference(resolution, [], [x7c641081_b821_11ec_a9ea_0f4925154247, x7c64115f_b821_11ec_a9ea_0f4925154247])).
5 9
